#### Duration: nearly instant

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import sys
import logging
logging.basicConfig(level=logging.INFO)
print(sys.executable)

/n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


In [3]:
from multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D import Inferencer2D

INFO:multicamera_airflow_pipeline.tim_240731.keypoints.predict_2D:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


### Get recording info (google sheets)

In [4]:
import requests
import pandas as pd
from io import BytesIO

In [5]:
# spreadsheet_url = 'https://docs.google.com/spreadsheet/ccc?key=14HIqUaSl_n-91hpAvmACY_iVY9nLKdlA6qklhxfZon0&output=csv&gid=0'
spreadsheet_url = "https://docs.google.com/spreadsheet/ccc?key=1jACsUmxuJ9Une59qmvzZGc1qXezKhKzD1zho2sEfcrU&output=csv&gid=0"
response = requests.get(spreadsheet_url)
recording_df = pd.read_csv(BytesIO(response.content))

In [6]:
recording_df[:3]

,Subject,duration_m,video_recording_id,ephys_id,calibration_id,video_location_on_o2,ephys_location_on_o2,calibration_location_on_o2,samplerate,username,n_ephys_streams,max_video_duration_m,use_local,User
0,M04002,10,24-05-01-13-26-43-110846,2024-05-01_13-26-37,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10,False,NaN
1,M04003,30,24-05-01-13-51-17-581428,2024-05-01_13-51-09,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10,False,NaN
2,M04002,30,24-05-01-15-37-07-757018,2024-05-01_15-36-57,24-05-01-13-45-07-825493,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,/n/groups/datta/tim_sainburg/datasets/chronic2...,150,tis697,1,10,False,NaN


### Run sync

In [7]:
for idx, recording_row in recording_df.iterrows():
    break

In [9]:
output_directory = Path("/n/groups/datta/tim_sainburg/datasets/scratch/") / "240806-3d-pipeline"

In [10]:
output_directory_camera_sync = output_directory / 'camera_sync' / recording_row.video_recording_id
output_directory_camera_sync.mkdir(parents=True, exist_ok=True)

In [11]:
from multicamera_airflow_pipeline.tim_240731.sync.sync_cameras import CameraSynchronizer

INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:Python interpreter binary location: /n/groups/datta/tim_sainburg/conda_envs/peromoseq/bin/python3


In [12]:
video_directory = Path(recording_row.video_location_on_o2)

TypeError: unsupported operand type(s) for /: 'str' and 'str'

In [ ]:
synchronizer = CameraSynchronizer(
    recording_directory = video_directory,
    output_directory = output_directory_camera_sync,
    SAMPLERATE = SAMPLERATE, # camera sample rate
    TRIGGER_PIN = 2, # Which pin camera trigger was on
)

In [26]:
synchronizer.run()

INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:Loading video config, metadata, and triggerdata
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: BackBottom
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: BackLeft
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: BackRight
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: FrontBottom
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: FrontLeft
INFO:multicamera_airflow_pipeline.tim_240731.sync.sync_cameras:	 aligning frames for camera: FrontRight


In [27]:
!ls {output_directory_camera_sync}

camera_sync.csv


In [30]:
!head {output_directory_camera_sync/'camera_sync.csv'}

,trigger_times,trigger_states,BackBottom_frames,BackLeft_frames,BackRight_frames,FrontBottom_frames,FrontLeft_frames,FrontRight_frames,datetime_est
0,0.0,0,0,0,0,0,0,0,1714584403.110846
1,6666.0,0,1,1,1,1,1,1,1714584403.117512
2,13332.0,1,2,2,2,2,2,2,1714584403.124178
3,19998.0,1,3,3,3,3,3,3,1714584403.130844
4,26664.0,1,4,4,4,4,4,4,1714584403.13751
5,33330.0,0,5,5,5,5,5,5,1714584403.144176
6,39996.0,0,6,6,6,6,6,6,1714584403.150842
7,46662.0,0,7,7,7,7,7,7,1714584403.157508
8,53328.0,0,8,8,8,8,8,8,1714584403.164174


In [29]:
pd.read_csv(output_directory_camera_sync/'camera_sync.csv')[:3]

,Unnamed: 0,trigger_times,trigger_states,BackBottom_frames,BackLeft_frames,BackRight_frames,FrontBottom_frames,FrontLeft_frames,FrontRight_frames,datetime_est
0,0,0.0,0,0,0,0,0,0,0,1.714584e+09
1,1,6666.0,0,1,1,1,1,1,1,1.714584e+09
2,2,13332.0,1,2,2,2,2,2,2,1.714584e+09
